In [2]:
#!/usr/local/bin/python3

# ---------------------------------------------------------------------------------------------------------------------
# IMPORTS
# ---------------------------------------------------------------------------------------------------------------------
# TODO organize these
import numpy as np
from scipy import interpolate
import pandas as pd
import copy

import os
import subprocess

from netCDF4 import Dataset
from wrf import to_np, getvar, smooth2d, get_basemap, latlon_coords, interplevel
import wrf

import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
import matplotlib.colors as colors
from matplotlib.colors import PowerNorm
from mpl_toolkits.basemap import Basemap, cm
import gdal

import os
import os.path
import configparser

from pathlib import Path

import optparse

# ---------------------------------------------------------------------------------------------------------------------
# CONFIGURATION and INITIALIZATION
# ---------------------------------------------------------------------------------------------------------------------

# Read in the config file
config = configparser.ConfigParser()
config.read('config.ini')

#pd.set_option('display.height', 1000)
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

# Close all previous plots
plt.close("all")

#%matplotlib inline

# # define command line arguments
# p = optparse.OptionParser()
# p.add_option('--datetime','-d')
# options,arguments = p.parse_args()

# data and shapefile directory and corresponding filenames
DEM_DIR = config['DEFAULT']['DEM_DIR']
data_file = os.path.join(DEM_DIR,'NWAC_Datalogger.csv')
var_file  = os.path.join(DEM_DIR,'NWAC_Datalogger_variables.csv')
dem       = os.path.join(DEM_DIR,'NWAC_lanczos_hillshade.tif')
#dem       = os.path.join(DEM_DIR,'merged_resampled_lanczos_hillshade.tif')

# wrf input dir and output image dir (need to add YYYYMMDDHH to both)
WRF_DIR = config['DEFAULT']['WRF_DIR']
IMAGE_DIR = config['DEFAULT']['IMAGE_DIR']

# Select the initialization date
ymdh = 2019061312

# Append the initialization date to create an output dir
outdir = os.path.join(IMAGE_DIR,str(ymdh))
if not os.path.isdir(outdir):
    os.makedirs(outdir)
    
# Forecast hours to process    
fh_start = 24  # this is the end of the hour
fh_end = 24    # this is the end of the hour

# Choose which WRF domain to run (3 = 4-km; 4 = 1.33-km)
# And adjust the output point spacing accordingly
nest = 3
sub = False     # true if the domain is smaller than "nwac"
if (sub==True):
    if (nest==3):
        points = 2
    if (nest==4):
        points = 6
else:
    if (nest==3):
        points = 6
    if (nest==4):
        points = 18

## get ymdh from command line options
#ymdh = options.datetime
#if ymdh == None:
#  print("Must provide datetime, --datetime YYYYMMDDHH option. Check usage")
#  exit()


class nf(float):
    def __repr__(self):
        str = '%.1f' % (self.__float__(),)
        if str[-1] == '0':
            return '%.0f' % self.__float__()
        else:
            return '%.1f' % self.__float__()

to_m      = lambda z_ft: .3048*z_ft             # converts ft to m
to_ft      = lambda z_m: 3.28084*z_m 
to_mph    = lambda spd_mph: 2.23694*spd_mph     # converts from m/s to mph
to_F      = lambda deg_C: (deg_C * 1.8) + 32
K_to_F    = lambda deg_K: ((deg_K-273.15) * 1.8) + 32
mm_to_in = lambda mm: mm*.0393701               # convert from mm to in

# ---------------------------------------------------------------------------------------------------------------------
# METHODS
# ---------------------------------------------------------------------------------------------------------------------
# Create the background map

def create_bm_fig(lat_lims=(43.8, 49.38), lon_lims=(-126.2,-119), 
                        figsize=(16,16), hillshade=False, dem_path='', domain=''):
    fig = plt.figure(figsize=figsize)
    ax = fig.add_axes([0.1,0.1,0.8,0.8])

    m = Basemap(projection='stere',lat_0 = np.mean(lat_lims), lon_0 = np.mean(lon_lims),\
                llcrnrlat=lat_lims[0],urcrnrlat=lat_lims[1],\
                llcrnrlon=lon_lims[0],urcrnrlon=lon_lims[1],\
                rsphere=6371200.,resolution='h')

    if hillshade:
        gdata = gdal.Open(dem_path)
        geo = gdata.GetGeoTransform()

        data = gdata.ReadAsArray()
        
        print(geo)

        xres = geo[1]
        yres = geo[5]

        xmin = geo[0] + xres * 0.5
        xmax = geo[0] + (xres * gdata.RasterXSize) - xres * 0.5
        ymin = geo[3] + (yres * gdata.RasterYSize) + yres * 0.5
        ymax = geo[3] - yres * 0.5
        stride = 1
        x, y = np.mgrid[xmin:xmax+xres*stride:xres*stride, ymax+yres*stride:ymin:yres*stride]
        x, y = m(x, y)
        
        ny, nx = data.shape
        if x.shape[0] > nx:
            x, y = x[:-1, :], y[:-1, :]
        if y.shape[1] > ny:
            x, y = x[:, :-1], y[:, :-1]
        m.pcolormesh(x, y, data.T[::stride,::stride], cmap='gray', ax=ax);
        

    m.drawcoastlines(linewidth=2);
    m.drawstates(linewidth=2);
    m.drawcountries(linewidth=2);
    m.drawcounties(linewidth=2, zorder=20);
    m.drawrivers(linewidth=2);
    
#    m.readshapefile('/Users/robertshahn/Documents/python-projects/ForFireLabs/roads/roadtrl010g_north_cascades',
#                   'roadtrl010g_north_cascades', linewidth=2, zorder=3, color ="red")
    
#     m.readshapefile('/Users/robertshahn/Documents/python-projects/ForFireLabs/roads/NatHwySysState',
#                   'NatHwySysState', linewidth=2, zorder=3, color =[.3,.3,.3])

    return m, fig, ax

# ---------------------------------------------------------------------------------------------------------------------
# SCRIPT BODY
# ---------------------------------------------------------------------------------------------------------------------
# TODO Remove various commented out scratch code.

for fh in range(fh_start,fh_end+1,1):
    print("We are starting FH "+str(fh))

    ncfile = Dataset(str(WRF_DIR)+'/'+str(ymdh)+'/wrfout_d'+str(nest)+'.'+str(ymdh)+'.f'+str(fh).zfill(2)+'.0000')
    fcst_time = to_np(wrf.times.get_times(ncfile))
    tstr = pd.to_datetime(fcst_time).strftime('%Y-%m-%d %H:%M')
    file_tstr = pd.to_datetime(fcst_time).strftime('%Y%m%d%H%M')
    var_df = pd.read_csv(var_file)

    if (fh == fh_start):
        # logger info
        var_df = pd.read_csv(var_file)
        logger_df = pd.read_csv(data_file, skiprows=[1])
        logger_df.dropna(how='all', axis=1, inplace=True)

        #Filter out stations with precip
        filt = var_df['Column Header'].str.contains('Precipitation')
        prec_stations = var_df[filt]['Datalogger_Name'].values
        filt = var_df['Column Header'].str.contains('Temperature')
        temp_stations = var_df[filt]['Datalogger_Name'].values

        filt = logger_df['NWAC Datalogger_name'].isin(temp_stations)
        latlon_df_temp = logger_df[filt][['NWAC Datalogger_name','Logger ID#','lat','lon','elevation']]\
        
        #TODO: Pair this down to stations of interest
        latlon_df_temp=latlon_df_temp.reindex([12,19,20,36,33,37,38,25,0,1,2,30,31,32, \
                                       8,9,10,39,40,6,7,26,5,46,48, \
                                       24,44,45,15,11,13,3,43,14,4,16,17,18, \
                                       21,22,23,41,42,28,29])
#         latlon_df_temp = latlon_df_temp.iloc[[0,1]]

#         filt = logger_df['NWAC Datalogger_name'].isin(prec_stations)
#         latlon_df_prec = logger_df[filt][['NWAC Datalogger_name','Logger ID#','lat','lon','elevation']]
#         latlon_df_prec=latlon_df_prec.reindex([12,18,35,33,0,29,8,6,25,47,23, \
#                                                43,14,13,3,4,16,20,40,27])

    if 'var_list' in locals():
        del var_list
    var_list = []
    #Get WRF fields
    #    fields = ['PREC_ACC_NC','SNOW_ACC_NC','ua','va','pressure','z','tc',"T2","U10","V10"]
    fields = ['T2','U10','V10','HGT']
    for field in fields:
         var_list.append(getvar(ncfile, field))

    #NOW PLOT T2 TEMP AND U10,V10

    def graph_temp_sfc(figsize=(16,16), lat_lims='', lon_lims='', 
                     hillshade='', dem_path='',domain='', 
                     spx='',alpha=''):
        m, fig, ax = create_bm_fig(figsize=figsize, lat_lims=lat_lims, lon_lims=lon_lims, 
                            hillshade=hillshade, dem_path=dem_path)

        
        #Make hot and cold colormaps
        colors1 = plt.cm.gist_ncar_r(np.linspace(0, 0.17, 30))
        colors2 = plt.cm.terrain(np.linspace(0., .47, 50))
        colors3 = plt.cm.seismic(np.linspace(0.52, 1, 36))
        colors4 = plt.cm.terrain_r(np.linspace(0.25, 0.53, 25))

        # combine them and build a new colormap
        combined = np.vstack((colors1, colors2, colors3, colors4))
        mymap = colors.LinearSegmentedColormap.from_list('my_colormap', combined)

        U10 = to_mph(var_list[1])  #interplevel(u, z, to_m(height))
        V10 = to_mph(var_list[2])  #interplevel(v, z, to_m(height))
        
        wrf_lat, wrf_lon = latlon_coords(var_list[0])   #grab the wrf lats and lons
        x, y = m(to_np(wrf_lon),to_np(wrf_lat))
        wrf_lat_b, wrf_lon_b = latlon_coords(U10)
        xb, yb = m(to_np(wrf_lon_b), to_np(wrf_lat_b))
        cf = m.contourf(x, y, K_to_F(to_np(var_list[0])), np.arange(-10, 64, 2), cmap=mymap, alpha=alpha)
        cs = m.contour(x, y, K_to_F(to_np(var_list[0])), range(-8,72,8), colors='k', alpha=0.3)
        
        #add wind barbs
        m.barbs(xb[::spx,::spx], yb[::spx,::spx], U10[::spx,::spx], V10[::spx,::spx])
        
        
        cs.levels = [nf(val) for val in cs.levels]

        fmt = '%r'
        plt.clabel(cs, cs.levels, colors='k', inline=True, fmt=fmt, fontsize=16)
        cbar = m.colorbar(cf,location='bottom',pad="5%")

        filt = lat_lims[1]>=latlon_df_temp['lat']
        sfcstns = latlon_df_temp[filt][['NWAC Datalogger_name','lat','lon']]
        filt = lat_lims[0]<=sfcstns['lat']
        sfcstns = sfcstns[filt]
        filt = lon_lims[0]<=sfcstns['lon']
        sfcstns = sfcstns[filt]
        filt = lon_lims[1]>=sfcstns['lon']
        latlon_df_sfcstns = sfcstns[filt]
        lat1, lon1 = latlon_df_sfcstns['lat'].values, latlon_df_sfcstns['lon'].values
        print("latl, lon1 = " + str(lat1[0]) + " " + str(lon1[0]))
#        print(type(lat1))
        x1, y1 = m(to_np(lon1), to_np(lat1))
        print("x1, y1 = " + str(x1[0]) + " " + str(y1[0]))
        x_y = to_np(wrf.ll_to_xy(ncfile, lat1[:], lon1[:]))
        yy=to_np(x_y[1,:])
        xx=to_np(x_y[0,:])
        print("yy, xx = " + str(yy[1]) + ", " + str(xx[0]))
        
        print(str(to_np(wrf.xy_to_ll(ncfile, xx[0], yy[0]))))
#        print(yy)
        scale = len(yy)*25
#        print("scale = " + str(scale))
        lat2 = np.zeros(scale)
        lon2 = np.zeros(scale)
        x2 = np.zeros(scale, dtype = int)
        y2 = np.zeros(scale, dtype = int)
        stn_assoc = []
        q = 0
        # This code makes the 25-point grid and gives each point a lat lon and associates it with a Temp station.
        for j in range(0,len(xx),1):
#             print(type(yy[j]))
            c = 0
            for a in range(-2,3,1):
                for b in range(-2,3,1):
                    lat2[q],lon2[q] = to_np(wrf.xy_to_ll(ncfile, xx[j]+a, yy[j]+b))
                    x2[q] = int(xx[j]+a)
                    y2[q] = int(yy[j]+b)
                    c=c+1
                    q=q+1
                    stn_assoc.append(logger_df['NWAC Datalogger_name'][j])
        print("stn_assoc = " + str(stn_assoc))
#        print(x3,y3)
        x1a, y1a = m(to_np(lon2), to_np(lat2))
        sfctemp = np.zeros((len(lat2),),dtype=float)
        sfcelev = np.zeros((len(lat2),),dtype=float)
        for i in range(0,len(y2),1):
        #    if not (np.isnan(tempvar[xx[i],yy[i]])):
            tempvar = round(float(to_np(K_to_F(var_list[0][y2[i],x2[i]]))),3)
            elev = round(float(var_list[3][y2[i],x2[i]]),1)
            sfctemp[i] = tempvar
            sfcelev[i] = elev
            if (domain == 'nwac'):
                m.scatter(x1a, y1a, s=100, marker='*', color='magenta', zorder=10, latlon=False)
            else:
                ax.text(x1a[i],y1a[i],tempvar,ha='center',va='center')

        lbl_fnt_sz = 16
        ttl_fnt_sz = 36
        cbar.set_label('Deg. F', size=lbl_fnt_sz)
        cbar.ax.tick_params(labelsize=lbl_fnt_sz) 
        plt.title('Sfc Temp & Wind {0} UTC'.format(tstr), fontsize=ttl_fnt_sz)
        fig.savefig(outdir+'/'+domain+'_sfc_temp_wind_{0}.png'.format(file_tstr), dpi=200)

        return sfctemp, latlon_df_sfcstns, sfcelev, y2, x2, lat2, lon2, scale
        # Skip plot generation if so specified.

    sfctemp, latlon_df_sfcstns, sfcelev, y2, x2, lat2, lon2, scale = graph_temp_sfc(figsize=(16,16), lat_lims=(43.8, 49.38), 
                    lon_lims=(-126.2,-119), hillshade=True, dem_path=dem, domain='nwac',
                    spx=points, alpha=.5)
    

We are starting FH 24
               NWAC Datalogger_name  Logger ID#        lat         lon  elevation
12                  Hurricane Ridge        81.0  47.970000 -123.499333     5250.0
19       Mt Baker - Heather Meadows       355.0  48.863000 -121.677850     4210.0
20              Mt Baker - Pan Dome       356.0  48.853000 -121.677200     5020.0
36      Stevens Pass - Schmidt Haus        13.0  47.746000 -121.092633     3950.0
33       Stevens Pass - Brooks Wind        90.0  47.738000 -121.106617     4850.0
37           Stevens Pass - Skyline        89.0  47.734000 -121.108100     5250.0
38          Stevens Pass - Tye Mill       101.0  47.732000 -121.085300     5180.0
25                    Mt Washington       360.0  47.426000 -121.699383     4340.0
0                     Alpental Base       121.0  47.444000 -121.424850     3100.0
1             Alpental Mid-Mountain        22.0  47.434000 -121.434850     4350.0
2                   Alpental Summit        23.0  47.439000 -121.442617     5

j = 25
j = 25
j = 25
j = 25
j = 25
j = 25
j = 26
j = 26
j = 26
j = 26
j = 26
j = 26
j = 26
j = 26
j = 26
j = 26
j = 26
j = 26
j = 26
j = 26
j = 26
j = 26
j = 26
j = 26
j = 26
j = 26
j = 26
j = 26
j = 26
j = 26
j = 26
j = 27
j = 27
j = 27
j = 27
j = 27
j = 27
j = 27
j = 27
j = 27
j = 27
j = 27
j = 27
j = 27
j = 27
j = 27
j = 27
j = 27
j = 27
j = 27
j = 27
j = 27
j = 27
j = 27
j = 27
j = 27
j = 28
j = 28
j = 28
j = 28
j = 28
j = 28
j = 28
j = 28
j = 28
j = 28
j = 28
j = 28
j = 28
j = 28
j = 28
j = 28
j = 28
j = 28
j = 28
j = 28
j = 28
j = 28
j = 28
j = 28
j = 28
j = 29
j = 29
j = 29
j = 29
j = 29
j = 29
j = 29
j = 29
j = 29
j = 29
j = 29
j = 29
j = 29
j = 29
j = 29
j = 29
j = 29
j = 29
j = 29
j = 29
j = 29
j = 29
j = 29
j = 29
j = 29
j = 30
j = 30
j = 30
j = 30
j = 30
j = 30
j = 30
j = 30
j = 30
j = 30
j = 30
j = 30
j = 30
j = 30
j = 30
j = 30
j = 30
j = 30
j = 30
j = 30
j = 30
j = 30
j = 30
j = 30
j = 30
j = 31
j = 31
j = 31
j = 31
j = 31
j = 31
j = 31
j = 31
j = 31
j = 31
j = 31
j = 31

In [ ]:
sfcstns_25pt = copy.deepcopy(latlon_df_sfcstns)
# for r in sfcstns_25pt.index:
#     for s in range(0,25,1):
#         sfcstns_25pt = sfcstns_25pt.append(sfcstns_25pt.iloc[r], ignore_index=True)


In [ ]:
lon2

In [ ]:
sfctemp.shape

In [ ]:
sizes(sfcstns_25pt)

In [ ]:
lat2[48:60]

In [ ]:
sfcstns_25pt = sfcstns_25pt.drop(sfcstns_25pt.index[0:45])

In [ ]:
for u in sfcstns_25pt.index:
    print(u)
    sfcstns_25pt.loc[u,"lat"] = lat2[u-45]
    sfcstns_25pt.loc[u,"lon"] = lon2[u-45]
    

In [ ]:
sfcstns_25pt

In [ ]:
from scipy import interpolate
tck = interpolate.splrep(x, y, s=0)
xnew = np.arange(0, 7000, 1000)
ynew = interpolate.splev(xnew, tck, der=0)

In [ ]:
if (fh == fh_start):
    tsfctable = copy.deepcopy(sfcstns_25pt)
    tsfcfile = os.path.join(outdir,"tsfc.txt") 
print(sfctemp.astype(float))
tsfctable['ELEV'] = to_ft(sfcelev.astype(float))
tsfctable['FH'+str(fh)] = sfctemp.astype(float)
if (fh == fh_end):
    d = open(tsfcfile, 'w')
    d.write(str(round(tsfctable,3)))
    d.close()  